In [1]:
import hopsworks
import pandas as pd
import joblib
import os
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2


In [2]:
key=""
with open("api-key.txt", "r") as f:
    key = f.read().rstrip()
os.environ['HOPSWORKS_PROJECT']="deloitte"
os.environ['HOPSWORKS_HOST']="6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai"
os.environ['HOPSWORKS_API_KEY']=key

In [3]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai:443/p/119
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# change to 'version=2' to try out the feature view with transformers, instead of sklearn transformers
version=1
fv = fs.get_feature_view(name="loans_applicants", version=version)

In [5]:
fv.init_batch_scoring(training_dataset_version=1)

In [6]:
def retrieve_data(feature_view):
    batch_data = feature_view.get_batch_data()
    return batch_data

df = retrieve_data(fv)
df

2023-03-24 20:14:48,132 INFO: USE `deloitte_featurestore`
2023-03-24 20:14:48,612 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`loan_amnt` `loan_amnt`, `fg1`.`term` `term`, `fg1`.`int_rate` `int_rate`, `fg1`.`installment` `installment`, `fg1`.`sub_grade` `sub_grade`, `fg1`.`purpose` `purpose`, `fg1`.`zip_code` `zip_code`, `fg1`.`id` `join_pk_id`, `fg1`.`issue_d` `join_evt_issue_d`, `fg0`.`earliest_cr_line_year` `earliest_cr_line_year`, `fg0`.`home_ownership` `home_ownership`, `fg0`.`annual_inc` `annual_inc`, `fg0`.`verification_status` `verification_status`, `fg0`.`dti` `dti`, `fg0`.`open_acc` `open_acc`, `fg0`.`pub_rec` `pub_rec`, `fg0`.`revol_bal` `revol_bal`, `fg0`.`revol_util` `revol_util`, `fg0`.`total_acc` `total_acc`, `fg0`.`initial_list_status` `initial_list_status`, `fg0`.`application_type` `application_type`, `fg0`.`mort_acc` `mort_acc`, `fg0`.`pub_rec_bankruptcies` `pub_rec_bankruptcies`, RANK() OVER (PARTITION BY `fg1`.`id`, `fg1`.`issue_d` ORDER BY `fg0`.`earliest_

,loan_amnt,term,int_rate,installment,sub_grade,purpose,zip_code,earliest_cr_line_year,home_ownership,annual_inc,...,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,10000.0,36 months,11.44,329.48,B4,vacation,22690,1990,RENT,117000.0,...,26.24,16.0,0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0
1,8000.0,36 months,11.99,265.68,B5,debt_consolidation,0,2004,MORTGAGE,65000.0,...,22.05,17.0,0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0
2,15600.0,36 months,10.49,506.97,B3,credit_card,0,2007,RENT,43057.0,...,12.79,13.0,0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0
3,7200.0,36 months,6.49,220.65,A2,credit_card,0,2006,RENT,54000.0,...,2.60,6.0,0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0
4,24375.0,60 months,17.27,609.33,C5,credit_card,11650,1999,MORTGAGE,55000.0,...,33.95,13.0,0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395214,10000.0,60 months,10.99,217.38,B4,debt_consolidation,30723,2004,RENT,40000.0,...,15.63,6.0,0,1990.0,34.3,23.0,w,INDIVIDUAL,0.0,0.0
395215,21000.0,36 months,12.29,700.42,C1,debt_consolidation,0,2006,MORTGAGE,110000.0,...,21.45,6.0,0,43263.0,95.7,8.0,f,INDIVIDUAL,1.0,0.0
395216,5000.0,36 months,9.99,161.32,B1,debt_consolidation,70466,1997,RENT,56500.0,...,17.56,15.0,0,32704.0,66.9,23.0,f,INDIVIDUAL,0.0,0.0
395217,21000.0,60 months,15.31,503.02,C2,debt_consolidation,29597,1990,MORTGAGE,64000.0,...,15.88,9.0,0,15704.0,53.8,20.0,f,INDIVIDUAL,5.0,0.0


In [9]:
def get_model(project):
    mr = project.get_model_registry()
    model = mr.get_model("lending_model", version=10)
    model_dir = model.download()
    return joblib.load(model_dir + "/lending_model.pkl")

model = get_model(proj)

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
predictions = model.predict(df)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [11]:
np.unique(predictions, return_counts=True)

(array([0, 1]), array([ 37214, 358005]))